<a href="https://colab.research.google.com/github/KimEunChae9506/2024-smartfarm-AI-strongberry/blob/main/%ED%85%8C%EC%8A%A4%ED%8A%B8/%EA%B9%80%EC%9D%80%EC%B1%84_%EB%AA%A8%EB%8D%B8%EB%B3%84_%ED%85%8C%EC%8A%A4%ED%8A%B82(%EA%B7%B8%EB%A3%B9%ED%99%94).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#모델별 테스트2
#서버 : 코랩 GPU
#데이터 전처리 : frmDate 그룹화
#모델 4가지 테스트
  #LinearRegression
  #RandomForestRegressor
  #SVR
  #XGBRegressor

#문제 : 정식일 이후 16주 동안 단위면적당 최소 에너지 비용(누적 난반용 도시가스 사용량)으로 최대 수확량을 예측 할 수 있는 환경/생육 모델을 구해라

#전처리
 # => frmDist 를 그룹화.
 # => 최소 에너지 비용으로 최대 수확량을 구하므로 누적 난방 에너지 사용량의 최솟값(min()) 을 독립변수에 추가.

#결과
# 1. LinearRegression, RandomForestRegressor => 속도는 네 모델 다 1초이내.
# SVR은 결정계수 음수값.

In [15]:
import pandas as pd
import numpy as np
import csv
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.metrics import r2_score
import chardet

# 데이터 로드
file_path = 'dataset.csv'
data = pd.read_csv(file_path)

# 데이터 전처리
# 'frmDist' 열을 기준으로 데이터 그룹화 및 평균값 계산하여 'group_df'에 저장
group_df = data.groupby('frmDist').mean()

# 'frmDist' 열을 기준으로 데이터 다시 그룹화하고, 'outtrn_cumsum' 열에서 최솟값 계산하여 'group_outtrn_cumsum_df'에 저장
group_outtrn_cumsum_df = data.groupby('frmDist').min()[['HeatingEnergyUsage_cumsum']]

# 'group_df'에서 'HeatingEnergyUsage_cumsum'과 'date' 열 제거
group_df.drop(['HeatingEnergyUsage_cumsum', 'date'], axis=1, inplace=True)

# 'group_df'와 'group_outtrn_cumsum_df'를 합쳐 'merge_df' 데이터프레임 생성
merge_df = pd.concat([group_df, group_outtrn_cumsum_df], axis=1)

# 독립 변수(X)와 종속 변수(y) 분리
X = merge_df.drop('outtrn_cumsum', axis=1)
y = merge_df['outtrn_cumsum']

# 데이터 분할 (학습용 데이터와 테스트용 데이터)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 데이터 스케일링
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 모델 리스트
model = LinearRegression()

# 모델 성능 평가
model.fit(X_train_scaled, y_train)
y_train_pred = model.predict(X_train_scaled)
y_test_pred = model.predict(X_test_scaled)

train_r2 = r2_score(y_train, y_train_pred)
test_r2 = r2_score(y_test, y_test_pred)


# 결과 출력
print(f"  Training R2: {train_r2:.4f}")
print(f"  Test R2: {test_r2:.4f}\n")

  Training R2: 0.9998
  Test R2: 0.9994



In [14]:
import pandas as pd
import numpy as np
import csv
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.metrics import r2_score
import chardet

# 데이터 로드
file_path = 'dataset.csv'
data = pd.read_csv(file_path)

# 데이터 전처리
# 'frmDist' 열을 기준으로 데이터 그룹화 및 평균값 계산하여 'group_df'에 저장
group_df = data.groupby('frmDist').mean()

# 'frmDist' 열을 기준으로 데이터 다시 그룹화하고, 'HeatingEnergyUsage_cumsum' 열에서 최솟값 계산하여 'group_outtrn_cumsum_df'에 저장
group_outtrn_cumsum_df = data.groupby('frmDist').min()[['HeatingEnergyUsage_cumsum']]

# 'group_df'에서 'outtrn_cumsum'과 'date' 열 제거
group_df.drop(['HeatingEnergyUsage_cumsum', 'date'], axis=1, inplace=True)

# 'group_df'와 'group_outtrn_cumsum_df'를 합쳐 'merge_df' 데이터프레임 생성
merge_df = pd.concat([group_df, group_outtrn_cumsum_df], axis=1)

# 독립 변수(X)와 종속 변수(y) 분리
X = merge_df.drop('outtrn_cumsum', axis=1)
y = merge_df['outtrn_cumsum']

# 데이터 분할 (학습용 데이터와 테스트용 데이터)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 데이터 스케일링
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 모델 리스트
model = RandomForestRegressor(random_state=42)

# 모델 성능 평가
model.fit(X_train_scaled, y_train)
y_train_pred = model.predict(X_train_scaled)
y_test_pred = model.predict(X_test_scaled)

train_r2 = r2_score(y_train, y_train_pred)
test_r2 = r2_score(y_test, y_test_pred)


# 결과 출력
print(f"  Training R2: {train_r2:.4f}")
print(f"  Test R2: {test_r2:.4f}\n")

  Training R2: 1.0000
  Test R2: 0.9994



In [13]:
import pandas as pd
import numpy as np
import csv
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.metrics import r2_score
import chardet

# 데이터 로드
file_path = 'dataset.csv'
data = pd.read_csv(file_path)

# 데이터 전처리
# 'frmDist' 열을 기준으로 데이터 그룹화 및 평균값 계산하여 'group_df'에 저장
group_df = data.groupby('frmDist').mean()

# 'frmDist' 열을 기준으로 데이터 다시 그룹화하고, 'outtrn_cumsum' 열에서 최솟값 계산하여 'group_outtrn_cumsum_df'에 저장
group_outtrn_cumsum_df = data.groupby('frmDist').min()[['HeatingEnergyUsage_cumsum']]

# 'group_df'에서 'HeatingEnergyUsage_cumsum'과 'date' 열 제거
group_df.drop(['HeatingEnergyUsage_cumsum', 'date'], axis=1, inplace=True)

# 'group_df'와 'group_outtrn_cumsum_df'를 합쳐 'merge_df' 데이터프레임 생성
merge_df = pd.concat([group_df, group_outtrn_cumsum_df], axis=1)

# 독립 변수(X)와 종속 변수(y) 분리
X = merge_df.drop('outtrn_cumsum', axis=1)
y = merge_df['outtrn_cumsum']

# 데이터 분할 (학습용 데이터와 테스트용 데이터)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 데이터 스케일링
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 모델 리스트
model = SVR()

# 모델 성능 평가
model.fit(X_train_scaled, y_train)
y_train_pred = model.predict(X_train_scaled)
y_test_pred = model.predict(X_test_scaled)

train_r2 = r2_score(y_train, y_train_pred)
test_r2 = r2_score(y_test, y_test_pred)


# 결과 출력
print(f"  Training R2: {train_r2:.4f}")
print(f"  Test R2: {test_r2:.4f}\n")

  Training R2: 0.0029
  Test R2: -0.0056



In [12]:
import pandas as pd
import numpy as np
import csv
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.metrics import r2_score
import chardet

# 데이터 로드
file_path = 'dataset.csv'
data = pd.read_csv(file_path)

# 데이터 전처리
# 'frmDist' 열을 기준으로 데이터 그룹화 및 평균값 계산하여 'group_df'에 저장
group_df = data.groupby('frmDist').mean()

# 'frmDist' 열을 기준으로 데이터 다시 그룹화하고, 'outtrn_cumsum' 열에서 최솟값 계산하여 'group_outtrn_cumsum_df'에 저장
group_outtrn_cumsum_df = data.groupby('frmDist').min()[['HeatingEnergyUsage_cumsum']]

# 'group_df'에서 'HeatingEnergyUsage_cumsum'과 'date' 열 제거
group_df.drop(['HeatingEnergyUsage_cumsum', 'date'], axis=1, inplace=True)

# 'group_df'와 'group_outtrn_cumsum_df'를 합쳐 'merge_df' 데이터프레임 생성
merge_df = pd.concat([group_df, group_outtrn_cumsum_df], axis=1)

# 독립 변수(X)와 종속 변수(y) 분리
X = merge_df.drop('outtrn_cumsum', axis=1)
y = merge_df['outtrn_cumsum']

# 데이터 분할 (학습용 데이터와 테스트용 데이터)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 데이터 스케일링
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 모델 리스트
model = XGBRegressor(random_state=42)

# 모델 성능 평가
model.fit(X_train_scaled, y_train)
y_train_pred = model.predict(X_train_scaled)
y_test_pred = model.predict(X_test_scaled)

train_r2 = r2_score(y_train, y_train_pred)
test_r2 = r2_score(y_test, y_test_pred)


# 결과 출력
print(f"  Training R2: {train_r2:.4f}")
print(f"  Test R2: {test_r2:.4f}\n")

  Training R2: 1.0000
  Test R2: 0.9819

